##### Multi-Source RAG

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
# set the environment variables
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

Tools are utilities designed to be called by a model: their inputs are designed to be generated by models, and their outputs are designed to be passed back to models.

Wikipedia Tool : To search about the topic on Wikipedia

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=2000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [7]:
print(wiki_tool.invoke({"query":"Rohit Sharma"}))

Page: Rohit Sharma
Summary: Rohit Sharma (born 30 April 1987) is an Indian international cricketer who currently plays for and captains the India national cricket team in Test and ODI matches. He previously captained in T20 matches and in the country's 2024 T20 World Cup win after which he retired from T20s in June. He is considered as one of the best batsmen of his generation and one of the greatest white-ball opening batters of all time. 
He holds several batting records which famously include most runs in T20 Internationals, most sixes in international cricket, most double centuries in ODI cricket (3), most centuries at Cricket World Cups (7) and joint most hundreds in Twenty20 Internationals (5). He is the first player to score 5 T20I centuries. He also holds the world record for the highest individual score (264) in a One Day International (ODI) match and is the only player to have scored three double-centuries in ODIs and also holds the record for scoring most hundreds (five) in 

In [8]:
print(f"Name: {wiki_tool.name}")
print(f"Description: {wiki_tool.description}")
print(f"args schema: {wiki_tool.args}")
print(f"returns directly?: {wiki_tool.return_direct}")

Name: wikipedia
Description: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
args schema: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
returns directly?: False


Reteiver tool : Creating tool to retrieve information about langsmith, store it in the database

In [15]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs={'device': 'cpu'},encode_kwargs={'normalize_embeddings': False})

chroma_db = Chroma('chroma_vectorstore',embedding_function=embedding)
chroma_db.add_documents(documents=documents,embedding=embedding)

retriever=chroma_db.as_retriever()

In [16]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever=retriever,
                                     name="langsmith_search",
                                     description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

Arvix Tool : Create a tool which search the arvix for specific research paper or related thing

In [17]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=3,doc_content_chars_max=500)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [20]:
arxiv_tool.invoke({'query':"Give me the summary of the attention all you need research paper"})

'Published: 2023-09-19\nTitle: Attention Is Not All You Need Anymore\nAuthors: Zhe Chen\nSummary: In recent years, the popular Transformer architecture has achieved great\nsuccess in many application areas, including natural language processing and\ncomputer vision. Many existing works aim to reduce the computational and memory\ncomplexity of the self-attention mechanism in the Transformer by trading off\nperformance. However, performance is key for the continuing success of the\nTransformer. In this pap'

In [21]:
tools = [arxiv_tool,wiki_tool,retriever_tool]

In [25]:
# model
from langchain_groq import ChatGroq
model_id = "llama-3.3-70b-versatile"
model = ChatGroq(
    model=model_id,
    temperature=0.5,
    max_retries=2,
)

Agents

In [45]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

c:\Users\admin\anaconda3\envs\env_langchain_tutorial\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [27]:
model_with_tools = model.bind_tools(tools=tools)

In [44]:
from langchain_core.messages import HumanMessage
from langchain.agents import create_react_agent,AgentExecutor

In [47]:
agent = create_react_agent(llm=model,tools=tools,prompt=prompt)
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor.invoke({"input":"Langchain"})



> Entering new AgentExecutor chain...
To answer questions about Langchain, I need to consider what Langchain is and where I can find information about it. Langchain is a framework for building applications that utilize large language models, so I should look for information related to this topic.

Action: wikipedia
Action Input: LangchainPage: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.Thought: The Wikipedia page provides a general overview of LangChain, but it may not have the most up-to-date or detailed information. To get more specific information about LangChain, I should try searching for it on a platform that specializes in language models or AI.

Action: arxiv
Action Input: LangchainPu

{'input': 'Langchain',
 'output': 'LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It is a language model integration framework that can be used to build various applications, including document analysis and summarization, chatbots, and code analysis. LangChain integrates seamlessly with LangSmith, a platform for building production-grade LLM applications, and can be used with other frameworks such as LangGraph. LangChain provides a way to closely monitor and evaluate LLM applications, allowing developers to ship quickly and with confidence.'}